# Shor's algorithm implementation

this is a basic implementation of the shor's factoring algorithm to factorise a number into two of it's prime factors. 
Right now it has some limitations like there's an incomplete implementation of modular exponentiation, therefore the algorithm will not work for larger N. And the algorithm currently works for just numbers consisting of 2 prime numbers. The problem is just a simple syntax error in python and will be resolved pretty easily. 

In [91]:
from qiskit import QuantumCircuit, transpile
from numpy.random import randint
from math import gcd
from qiskit_aer import AerSimulator, Aer
import numpy as np


# Function to compute continued fractions and extract the period
def continued_fraction_expansion(value, max_denominator):
    fractions = []
    for _ in range(max_denominator):
        integer_part = int(value)
        fractions.append(integer_part)
        value -= integer_part
        if value == 0:
            break
        value = 1 / value
    # Compute the denominator
    denominator = 1
    for f in reversed(fractions):
        denominator = f * denominator + (1 if denominator == 0 else 0)
    return denominator


# QFT Implementation from Scratch
def qft_from_scratch(circuit, qubits):
    """Applies the QFT on the specified qubits in the circuit."""
    num_qubits = len(qubits)
    for i in range(num_qubits):
        # Apply Hadamard to the i-th qubit
        circuit.h(qubits[i])
        # Apply controlled phase gates
        for j in range(i + 1, num_qubits):
            angle = np.pi / (2 ** (j - i))
            circuit.cp(angle, qubits[j], qubits[i])
    # Swap qubits for correct output order
    for i in range(num_qubits // 2):
        circuit.swap(qubits[i], qubits[num_qubits - i - 1])


# Modular exponentiation circuit (simplified placeholder)
def mod_exp(a, N, t, n):
    """Placeholder for modular exponentiation circuit."""
    circuit = QuantumCircuit(t + n)
    # Apply dummy operations (actual implementation requires modular arithmetic)
    for i in range(t):
        circuit.x(n)  # Replace with controlled modular multiplication
    return circuit


# Shor's Algorithm Main Implementation
def shors_algorithm(N):
    # Step 1: Check if N is prime or even
    if N % 2 == 0:
        return 2  # Trivial factor
    if all(N % i != 0 for i in range(2, int(np.sqrt(N)) + 1)):
        return None  # N is prime

    while True:
        # Step 2: Choose a random number 'a'
        a = randint(2, N)
        g = gcd(a, N)
        if g > 1:
            return g  # Trivial factor found immediately

        # Step 3: Set up quantum phase estimation
        t = 2 * int(np.ceil(np.log2(N)))  # Number of counting qubits
        n = int(np.ceil(np.log2(N)))  # Number of modular exponentiation qubits
        circuit = QuantumCircuit(t + n, t)

        # Initialize counting qubits
        circuit.h(range(t))
        # Modular exponentiation
        mod_exp_circuit = mod_exp(a, N, t, n)
        circuit.compose(mod_exp_circuit, range(t + n), inplace=True)

        # Apply QFT from scratch to the counting qubits
        qft_from_scratch(circuit, range(t))

        # Measure the counting qubits
        circuit.measure(range(t), range(t))

        # Step 3: Run the quantum circuit
        simulator = AerSimulator()
        compiled_circuit = transpile(circuit, simulator)
        result = simulator.run(compiled_circuit).result()
        counts = result.get_counts()

        # Step 4: Interpret the measurement result
        measured_value = max(counts, key=counts.get)
        decimal_value = int(measured_value, 2) / (2 ** t)

        # Step 5: Use classical continued fractions to find the period
        r = continued_fraction_expansion(decimal_value, N)

        circuit.draw("mpl")

        # Step 6: Compute potential factors
        if r % 2 == 1 or pow(a, r // 2, N) == (N - 1):
            return None  # Failed to find a factor

        factor1 = gcd(pow(a, r // 2) - 1, N)
        factor2 = gcd(pow(a, r // 2) + 1, N)

        if factor1 > 1 and factor2 > 1:
                return factor1, factor2


# Example usage
N = 15  # Number to factorize
num = N
factors_list = []
while(N > 1):
    temp_factor = shors_algorithm(N)
    
    if temp_factor:
        factors_list.append(temp_factor)
        N = N / temp_factor
    else:
        factors_list.append(N)
        N = 1
# factors = shors_algorithm(N)
if factors_list:
    print(f"Factors of {num}: {factors_list}")
else:
    print("Failed to find factors.")


Factors of 15: [5, 3.0]
